In [5]:
import data_utils



['landcover/lulc-patches-pairs-32x32-102420.npy',
 'landcover/lulc-patches-pairs-32x32-103028.npy',
 'landcover/lulc-patches-pairs-32x32-108132.npy',
 'landcover/lulc-patches-pairs-32x32-110513.npy',
 'landcover/lulc-patches-pairs-32x32-112374.npy',
 'landcover/lulc-patches-pairs-32x32-11850.npy',
 'landcover/lulc-patches-pairs-32x32-121111.npy',
 'landcover/lulc-patches-pairs-32x32-128142.npy',
 'landcover/lulc-patches-pairs-32x32-131464.npy',
 'landcover/lulc-patches-pairs-32x32-141004.npy',
 'landcover/lulc-patches-pairs-32x32-141960.npy',
 'landcover/lulc-patches-pairs-32x32-1421.npy',
 'landcover/lulc-patches-pairs-32x32-142622.npy',
 'landcover/lulc-patches-pairs-32x32-144478.npy',
 'landcover/lulc-patches-pairs-32x32-145292.npy',
 'landcover/lulc-patches-pairs-32x32-145862.npy',
 'landcover/lulc-patches-pairs-32x32-146462.npy',
 'landcover/lulc-patches-pairs-32x32-149989.npy',
 'landcover/lulc-patches-pairs-32x32-151447.npy',
 'landcover/lulc-patches-pairs-32x32-151487.npy',
 'l

### Convert borders shapefile to geopackage

In [2]:
import geopandas as gpd

gdf = gpd.read_file('lql-data/misc/WB_countries_Admin0_10m/WB_countries_Admin0_10m.shp')
new_geoms = [gdf.unary_union]

world_gdf = gpd.GeoDataFrame(geometry=new_geoms, crs=gdf.crs)
world_gdf.to_file('lql-data/misc/world.gpkg', driver='GPKG')

CPLE_AppDefinedError: b'sqlite3_exec(CREATE TRIGGER "trigger_delete_feature_count_world" AFTER DELETE ON "world" BEGIN UPDATE gpkg_ogr_contents SET feature_count = feature_count - 1 WHERE lower(table_name) = lower(\'world\'); END;) failed: disk I/O error'

Exception ignored in: 'fiona.ogrext.gdal_flush_cache'
Traceback (most recent call last):
  File "fiona/_err.pyx", line 198, in fiona._err.GDALErrCtxManager.__exit__
fiona._err.CPLE_AppDefinedError: b'sqlite3_exec(CREATE TRIGGER "trigger_delete_feature_count_world" AFTER DELETE ON "world" BEGIN UPDATE gpkg_ogr_contents SET feature_count = feature_count - 1 WHERE lower(table_name) = lower(\'world\'); END;) failed: disk I/O error'
